In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


In [3]:
!pip install yargy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 110 kB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 24.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ab64ed12b9817e02d65161690dec890891898720890597eca963bbec2430ecc3
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [4]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, dictionary

In [5]:
insight = np.empty(len(df), np.ndarray)
insight.fill('greeting=False; goodbye=False; introduction=False')
gr_flg = np.zeros(len(df['dlg_id'].unique()))
gb_flg = np.zeros(len(df['dlg_id'].unique()))
manager = np.empty(len(df['dlg_id'].unique()), np.ndarray)
company = np.empty(len(df['dlg_id'].unique()), np.ndarray)

In [6]:
# Правила
# Приветствие
R1 = or_(rule(dictionary({'здравствуйте'}))
      ,rule(dictionary({'добрый'}), gram('NOUN')))
parser = Parser(R1) 

# Имя менеджера
R2 = or_(
    rule(dictionary({'меня'}), gram('Name')),
    rule(dictionary({'меня'}), dictionary({'зовут'}), gram('Name')),
    rule(dictionary({'это'}), gram('Name')))
parser2 = Parser(R2)

# Прощание
R3 = or_(rule(dictionary({'до'}), dictionary({'свидания'})), 
          rule(dictionary({'всего'}), dictionary({'хорошего'})), 
          rule(dictionary({'всего'}), dictionary({'доброго'})), 
          rule(dictionary({'хорошего'}), dictionary({'вечера'})))
parser3 = Parser(R3)

# Название компании
R4 = rule (dictionary({'компания'}), gram('NOUN'))
parser4 = Parser(R4)

In [7]:
for i, row in df[df['role'] == 'manager'].iterrows():
  for match in parser.findall(row['text']):
    t1 = [x.value for x in match.tokens]
    if len(t1) > 0:
      insight[i] = insight[i].replace('greeting=False', 'greeting=True')
      gr_flg[row['dlg_id']] = 1
  for match in parser2.findall(row['text']):
    t2 = [x.value for x in match.tokens]
    if len(t2) > 0:
      insight[i] = insight[i].replace('introduction=False', 'introduction=True')
      manager[row['dlg_id']] = t2[-1]
  for match in parser3.findall(row['text']):
    t3 = [x.value for x in match.tokens]
    if len(t3) > 0:
      insight[i] = insight[i].replace('goodbye=False', 'goodbye=True')
      gb_flg[row['dlg_id']] = 1
  for match in parser4.findall(row['text']):
    t4 = [x.value for x in match.tokens]
    if len(t4) > 0:
      company[row['dlg_id']] = t4[-1]

In [8]:
df['insight'] = insight
# для клиентов не заполняем:
df.loc[df['role'] == 'client', 'insight'] = ''

In [9]:
df

,dlg_id,line_n,role,text,insight
0,0,0,client,Алло,
1,0,1,manager,Алло здравствуйте,greeting=True; goodbye=False; introduction=False
2,0,2,client,Добрый день,
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,greeting=False; goodbye=False; introduction=True
4,0,4,client,Ага,
...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,greeting=False; goodbye=False; introduction=False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,greeting=False; goodbye=False; introduction=False
477,5,140,client,Спасибо спасибо,
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,


In [10]:
# информация про диалоги
df_info = pd.DataFrame({'dlg_id': range(len(df['dlg_id'].unique())), 'manager': manager, 'company': company})
df_info['check'] = gr_flg*gb_flg # проверка на то, что менеджер поздоровался и попрощался
df_info

,dlg_id,manager,company,check
0,0,ангелина,диджитал,1.0
1,1,ангелина,диджитал,1.0
2,2,ангелина,диджитал,1.0
3,3,максим,китобизнес,1.0
4,4,None,None,0.0
5,5,анастасия,None,0.0
